Aluno: Felipe Adrian Moreno Vera
Materia: Estadistica Computacional
Doutorado - EMAp

### Setting global variables and Libraries

In [1]:
# !pip install matplotlib
# !pip install numpy
# !pip install scipy
# !pip install gif
# !pip install seaborn
# !pip install pandas
# !pip install sklearn
# !pip install scikit-learn

In [2]:
import numpy as np
from scipy.stats import t, describe as stat_describe
from sklearn.metrics import mean_squared_error
from scipy.stats import invgamma, invwishart, invgauss, norminvgauss 

In [3]:
q = 13 # number of samples
n = 3 # dimensions of samples
N = n*q

## Paper 3: Blocked Gibbs sampling

The so-called Gibbs sampler is a work horse of Computational Statistics.  
It depends on decomposing a target distribution into conditional densities from which new values of a given coordinate can be drawn.

One of the difficulties one might encounter with the Gibbs sampler is that it might be slow to converge, specially in highly-correlated targets.  
In Statistics, multilevel models (also called hierarchical or random effects) are extremely useful in modelling data coming from stratified structures (e.g. individuals within a city and cities within a state) and typically present highly correlated posterior distributions.

One way to counteract the correlation between coordinates in the Gibbs sampler is to __block__ them together, and sample correlated coordinates jointly.

For this assigment you are referred to the 2009 _Journal of Computational and Graphical Statistics_ paper by Tan and Hobert.

* Precisely describe the so-called blocked Gibbs sampler;  
  __Hint:__ you do not need to describe theoretical properties of the algorithm given in this paper; a general description of the algorithm should suffice. 
  
__sol__:

In simple words, the main difference is that Blocked Gibbs Sampler groups two or more variables together (called __block__) and samples from their __joint distribution conditioned on all other variables__, rather than sampling from each one individually (Gibbs Sampler). 


* Explain the advantages -- both theoretical and practical -- of a clever blocking scheme;

__sol__:

The main advantage is that Block Gibbs Sampler ensure that the estimator is unbiased and the strong law of large numbers (SLLN) implies that it converges almost surely to $E_{\pi}g$; that is, it is also strongly consistent.

* Would it be possible to apply the "simple" Gibbs sampler in this example? Why?

__sol__:

#### Implementation:

The model described is $Y_{ij} = \theta_i + \epsilon_{ij}$, $i=1,\dots,q$, $j=1,\dots,n$. Where $\theta_1,\dots,\theta_q \sim \mathcal{N}(\mu, \sigma^2_{\theta})$, $\epsilon_{ij} \sim \mathcal{N}(0, \sigma^2_{\epsilon})$, and $(\mu, \sigma^2_{\theta}, \sigma^2_{\epsilon})$ are unknown parameter. So, we have the following equation:

\begin{equation}
\pi_{a,b}(\mu, \sigma^2_{\theta}, \sigma^2_{\epsilon}) = (\sigma^2_{\theta})^{-(a+1)} (\sigma^2_{\epsilon})^{-(b+1)}
\end{equation}

Where $a$ and $b$ are known hyper-parameters. We define as posteriori:

\begin{equation}
\pi(\theta, \mu, \sigma^2_{\theta}, \sigma^2_{\epsilon})  \propto f(y|\theta, \mu, \sigma^2_{\theta}, \sigma^2_{\epsilon})\, f(\theta|\mu, \sigma^2_{\theta}, \sigma^2_{\epsilon}) \, \pi_{a,b}(\mu, \sigma^2_{\theta}, \sigma^2_{\epsilon})
\end{equation}

Where:

\begin{equation}
f(y|\theta, \mu, \sigma^2_{\theta}, \sigma^2_{\epsilon}) = \prod^q_{i=1} \prod^{n_i}_{j=1} \frac{1}{\sqrt{2\pi \sigma^2_{\epsilon}}} \exp \{ -\frac{1}{2 \sigma^2_{\epsilon} } (y_{ij} - \mu )^2 \}
\end{equation}

And:

\begin{equation}
f(\theta|\mu, \sigma^2_{\theta}, \sigma^2_{\epsilon}) = \prod^q_{i=1} \frac{1}{\sqrt{2\pi \sigma^2_{\theta}}} \exp \{ -\frac{1}{2 \sigma^2_{\theta} } (\theta_i - \mu )^2 \}
\end{equation}


Folowwing the ideas, that function was replaced by:

\begin{equation}
f_y(\theta, \epsilon) = g(\theta, \mu, \sigma^2_{\theta}, \sigma^2_{\epsilon}) \, \pi(\theta, \mu, \sigma^2_{\theta}, \sigma^2_{\epsilon})
\end{equation}

Where $g()$ is our new function to estimate and $\pi()$ is our density function respectively, as was defined in paper. 

Results in Hobert and Casella (1996) show that the posterior is proper if and only if:

\begin{equation}
a<0\,\,,a+\frac{q}{2}>\frac{1}{2}\, \text{,  }\, a+b> \frac{1-N}{2}
\end{equation}

from this, we will take $a=-\frac{1}{2}$ and $b=0$ (also, they recommend this values in appendix).

In [4]:
a = -1/2
b = 0

In [5]:
def pi_func(sigma_theta, sigma_epsilon, a=-1/2, b=-1/2):
    return (sigma_theta)**(-1*(a+1)) * (sigma_epsilon)**(-1*(b+1))

In [6]:
mu_theta = 4.8
sigma_theta = 1/2
sigma_epsilon = 1/2

In [7]:
def get_mean_Y(Y):
    #n_i = Y.shape[1]
    #for i in range(n_i):
    #    Y[:,i] = Y[:,i]/(i+1)
    return np.sum(Y, axis=1)/Y.shape[1]

In [8]:
def generate_data(q, n, mu_theta, sigma_theta, sigma_epsilon):
    epsilon = np.random.normal(0.0, sigma_epsilon, size=(q,n))
    thetas = np.random.normal(mu_theta, sigma_theta, size=(q,1))
    return thetas, epsilon

We got a similar mean of original data in section 5.

In [24]:
thetas, epsilon = generate_data(q, n, mu_theta, sigma_theta, sigma_epsilon)
Y = thetas + epsilon
Y

array([[5.25433197, 5.15599977, 5.23662981],
       [4.76206512, 5.15618728, 5.21902361],
       [4.20455041, 4.25723101, 4.06970786],
       [3.90264106, 3.70531302, 4.14149953],
       [3.89815449, 4.26711131, 3.96631   ],
       [5.33475761, 3.96883007, 4.48563916],
       [5.71629972, 4.59558319, 5.98629365],
       [5.08455474, 5.15645604, 5.38060025],
       [4.46603545, 3.76986216, 4.23073778],
       [4.61242925, 5.12904952, 3.78958416],
       [3.93388431, 4.33176795, 4.18369523],
       [4.56167101, 4.17484775, 4.61699277],
       [4.25733711, 4.07386335, 3.76803068]])

In [25]:
Y_i = get_mean_Y(Y)
Y_i

array([5.21565385, 5.04575867, 4.17716309, 3.91648454, 4.0438586 ,
       4.59640895, 5.43272552, 5.20720368, 4.15554513, 4.51035431,
       4.1497825 , 4.45117051, 4.03307704])

We calculate the global mean:

\begin{equation}
\bar{y} = \frac{1}{N} \sum^q_{i=1} \sum^{n_i}_{j=1} y_{i,j}
\end{equation}

In [26]:
np.mean(Y), np.mean(thetas)

(4.533475875593315, 4.590711037067509)

Besides, the $SST$:

\begin{equation}
SST = n_i \sum^q_{i=1} (y_{i} - \bar{y} )^2
\end{equation}

Where:

\begin{equation}
y_{i} = \frac{1}{n_i} \sum_{j} y_{i,j}
\end{equation}

In [27]:
n*np.sum(np.square(get_mean_Y(Y) - np.mean(Y)))

9.868326920244009

Besides, the $SSE$:

\begin{equation}
SSE = \sum^q_{i=1} \sum^{n_i}_{j=1}(y_{i,j} - \bar{y}_{i} )^2
\end{equation}

In [31]:
sum_ = []
for i in range(Y.shape[0]):
    sum_.append(n*np.sum( np.square(Y[i,:] - Y_i[i]) ) )
np.sum(sum_)

11.666098300540602

* Implement the blocked Gibbs sampler discussed in the paper in order to fit the model of Section 1 to the data described in Section 5 therein.
    
 __sol__:

We use the gibbs sampling idea from previous work but changing some conditions: this block gibbs sampler is a two-variable Gibbs sampler that updates $\sigma^2 = (\sigma^2_{\theta}, \sigma^2_{\epsilon})$ and $\xi = (\mu, \theta)$, so in each step we have:

* Let's supose that we have the iteration $k$ as $(\sigma^2_k, \xi_k)$.
* One iteration updates $\sigma^2_{k+1}$ conditional on $\xi_k$.
* Then, updates $\xi_{k+1}$ conditional on $\sigma^2_{k+1}$.
* And so on.

So, if we sample $(\sigma^{2}_0, \xi_0), (\sigma^{2}_1, \xi_1), \dots\,$ we would estimate $E_{\pi}g$ using MonteCarlo:

\begin{equation}
\bar{g}_N = \frac{1}{N} \sum _{n=0} ^{N-1} g(\sigma^{2}_n, \xi_n)
\end{equation}

Also, we know that:

\begin{equation}
\pi(\theta, \mu, \sigma^2_{\theta}, \sigma^2_{\epsilon}) = \pi(\sigma^2_{\epsilon} | \theta) \, \pi(\mu | \theta) \, \, \pi(\sigma^2_{\theta}|\theta)
\end{equation}

Where:
* $\pi(\sigma^2_{\epsilon} | \theta)$ is an inverse gamma density.
* $\pi(\sigma^2_{\theta}|\theta)$ is an inverse gamma density.
* $\pi(\mu | \theta)$ is an inverse normal density.

Since we know that $\sigma^2_{\theta}$, $\sigma^2_{\epsilon}$, and $\xi$ are independent, we have:

\begin{equation}
\sigma^2_{\theta}|\xi \sim IG(\frac{q}{2} + a, \frac{1}{2} \sum_i (\theta_i - \mu))
\end{equation}

\begin{equation}
\sigma^2_{\epsilon}|\xi \sim IG(\frac{M}{2} + b, \frac{1}{2} \sum_{i,j} (y_{i,j} - \theta_i))
\end{equation}

\begin{equation}
\end{equation}

We know:

\begin{equation}
E(\mu | \sigma^2) = E(\mu | \sigma^2_{\theta}, \sigma^2_{\epsilon}) = \frac{1}{t} \sum^q_{i=1} \frac{n_i \bar{y}_i}{\sigma^2_{\epsilon} + n_i \sigma^2_{\theta}}
\end{equation}

Where:

\begin{equation}
t = \sum^q_{i=1} \frac{n_i}{\sigma^2_{\epsilon} + n_i \sigma^2_{\theta}}
\end{equation}

So, we can estimate $\mu$ using that.

For our implementation, we will use _thin_ which means __Thinning__. This consists in picking separated points from the sample, at each k-th step.

We will estimate $\mu$, $\sigma^2_{\theta}$, and $\sigma^2_{\epsilon}$:

In [14]:
def get_t(q, n, sigma_epsilon, sigma_theta):
    t = 0
    for i in range(q):
        t = t + n/(sigma_epsilon**2 + n*sigma_theta**2)
    return t

In [40]:
def gibbs_sampler(Y, thetas, mu_theta, a=-1/2, b=0, iterations=1000,thin=500):
    mat = np.zeros((iterations,3))
    n = Y.shape[1]
    q = Y.shape[0]
    M = q*n
    for i in range(iterations):
        # updates sigmas
        s_theta = np.sqrt(abs(invgauss.rvs( q/2 + a, 1/2*np.sum(thetas - mu_theta) ) ))
        s_epsilon = np.sqrt(abs(invgauss.rvs(M/2 + b, 1/2*np.sum(Y - thetas) ) ))
        # updates mu
        t = get_t(q, M, s_epsilon, s_theta)
        Y_i = get_mean_Y(Y)
        mu_theta = 1/t*np.sum( M*Y_i/(s_epsilon**2 + M*s_theta**2) )
        #mu_theta = np.mean(thetas)
        #thetas, eps = generate_data(q, n, mu_theta, s_theta, s_epsilon)
        #Y = thetas + eps
        mat[i] = [mu_theta, s_theta, s_epsilon]
    return mat

In [77]:
def block_gibbs_sampler(q=13, n=3, mu_theta=4.8, sigma_theta=1/2, sigma_epsilon=1/2, a=-1/2, b=0, iterations = 1000):
    
    thetas, epsilon = generate_data(q, n, mu_theta, sigma_theta, sigma_epsilon)
    Y_original = thetas + epsilon
    # gibbs sampler
    estimators = gibbs_sampler(Y_original, thetas, mu_theta, a=-1/2, b=0, iterations=iterations)
    best = np.mean(np.square(estimators - np.array([mu_theta, sigma_theta, sigma_epsilon])), axis=1)
    return estimators[np.where(best == best.min())][0]

For data configuration, we define the following varaibles:  
__note__ that with that configurations, the Block Gibbs Sampler should be __geometrically ergodic__.

* Assess convergence (or lack thereof) and mixing of the resulting chain.

__sol__:

In [85]:
mu_theta_new, sigma_theta_new, sigma_epsilon_new = block_gibbs_sampler()
thetas_generated, epsilon_generated = generate_data(q, n, mu_theta_new, sigma_theta_new, sigma_epsilon_new)
Y_generated = thetas_generated + epsilon_generated

In [86]:
np.mean(Y_generated), np.mean(thetas_generated)

(4.664951291550561, 4.673353982411732)

$SST$

In [87]:
n*np.sum(np.square(get_mean_Y(Y) - np.mean(Y)))

9.868326920244009

$SSE$

In [88]:
sum_ = []
for i in range(Y.shape[0]):
    sum_.append(n*np.sum( np.square(Y[i,:] - Y_i[i]) ) )
np.sum(sum_)

11.666098300540602

* Confirm your results agree with those given by the original authors up to Monte Carlo error.

__sol__:

Our new estimator $\tilde{g}_{R}$ derivated from $\bar{g}_N$ is defiend as:

\begin{equation}
\bar{g}_{N} = \frac{1}{N} \sum^{N-1}_{i=0} g(X_n) = \frac{1}{N} \sum^{N-1}_{i=0} g(\sigma^{2}_n, \xi_n)
\end{equation}

Becomes:

\begin{equation}
\tilde{g}_{R} = \frac{\sum^R_{t=1} S_t}{\sum^R_{t=1} N_t}
\end{equation}

Where:

\begin{equation}
S_t = \frac{\sum^R_{t=1} S_t}{\sum^R_{t=1} N_t}
\end{equation}

Y

* Comment on the significance of geometric ergodicity for the blocked Gibbs sampler proposed by Tan-2009.

__sol__:

If an estimator is __geometrically ergodicity__ and there is $\alpha>0$ such that $E_{\pi}|g|^{2+\alpha}<\infty$. Where $E_{\pi}g$ is estimated by using the classical Monte Carlo.  
So, in Block Gibbs sampling for a high value N of samples and dimension $q\geq2$ we can calculate an asymptotic standard error for $\bar{g}_N$. This means knowing some initial conditions, such as minimal number of samples to converge and minimal dimension data. Besides, the article proof that for that conditions (such as $q\geq4$ and $M\geq q+3$) the block gibbs is a strongly estimator.